In [ ]:
from pyspark.sql import SparkSession
import json,time,os
from pyspark.sql.functions import col,to_timestamp,unix_timestamp,expr

if __name__ == "__main__":
    # file_path_sec = "/opt/spark/data/xcom_data.json"
    file_path_sec = "/home/aky/data/xcom_data.json"
    if os.path.exists(file_path_sec):
        print("################# 존재합니다. spark 시작합니다 #################")
        spark = SparkSession.builder.appName("DataProcessing") \
                .getOrCreate()
                        
        with open(file_path_sec, 'r') as f:
            json_data = json.load(f)
            data = []
            for i in json_data:
                i_replace = i.replace("'",'"')
                data.append(json.loads(i_replace))
            df = spark.createDataFrame(data)
            df = df.withColumn("order_delivered_customer_date", to_timestamp(col("order_delivered_customer_date"))) \
                    .withColumn("order_purchase_timestamp", to_timestamp(col("order_purchase_timestamp"))).withColumn("time_diff_seconds", unix_timestamp(col("order_delivered_customer_date")) - unix_timestamp(col("order_purchase_timestamp"))).withColumn(
                "formatted_time",
                expr("concat(floor(time_diff_seconds / 86400), 'd ', floor((time_diff_seconds % 86400) / 3600), 'h ')")
            )
            
            df = df.drop("id","order_estimated_delivery_date","order_approved_at","order_delivered_carrier_date")
            df.show()
            spark.stop()
    else:
        print("################# 존재하지 않습니다 #################")


################# 존재합니다. spark 시작합니다 #################
+--------------------+-----------------------------+--------------------+------------------------+------------+-----------------+--------------+
|         customer_id|order_delivered_customer_date|            order_id|order_purchase_timestamp|order_status|time_diff_seconds|formatted_time|
+--------------------+-----------------------------+--------------------+------------------------+------------+-----------------+--------------+
|e7e6da2ddce69ab0b...|          2017-01-16 15:53:34|5965b589a33443dd1...|     2017-01-05 13:36:41|   delivered|           958613|       11d 2h |
|ed0b5e13c101afa09...|          2017-01-27 13:16:00|bb448c2c512a6883a...|     2017-01-20 11:38:56|   delivered|           610624|        7d 1h |
|5156283c24fef4abe...|          2017-02-09 15:44:10|c3ad507aba1f6b473...|     2017-01-21 19:46:23|   delivered|          1627067|      18d 19h |
|b5ea05c72a89dbe10...|          2017-01-28 04:04:57|32339685fefe9062d...|  